In [1]:
push!(LOAD_PATH,".")

4-element Vector{String}:
 "@"
 "@v#.#"
 "@stdlib"
 "."

# Import CHAKRA (and an implementation)

In [2]:
using Chakra, ChakraImp

# Define an attribute type
The following defines a type pc of pitch classes and associates it an attribute name ":pc"

In [3]:
abstract type PC end
struct pc <: PC
    val::Char
end
Chakra.@associatedType(:pc,PC)

# Represent choral
The following represents the first 10 notes of a bach choral as a sequence of constituents with pitch class attributes.

In [4]:
e1 = set(delim(),:pc,pc('G'))
e2 = set(delim(),:pc,pc('G'))
e3 = set(delim(),:pc,pc('D'))
e4 = set(delim(),:pc,pc('B'))
e5 = set(delim(),:pc,pc('A'))
e6 = set(delim(),:pc,pc('G'))
e7 = set(delim(),:pc,pc('G'))
e8 = set(delim(),:pc,pc('A'))
e9 = set(delim(),:pc,pc('B'))
e10 = set(delim(),:pc,pc('A'))
choral = [e10,e9,e8,e7,e6,e5,e4,e3,e2,e1]

10-element Vector{ChakraImp.Obj}:
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('A')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('B')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('A')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('A')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('B')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('D')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))
 ChakraImp.Obj(ChakraImp.Id[], Dict{Symbol, Any}(:pc => pc('G')))

# Define viewpoint
The following defines an atomic viewpoint which projects out the pitch class of the last constituent of a sequence

In [5]:
using Viewpoints
v = AtomicViewpoint(pc,:pc)

AtomicViewpoint{PC}(pc, :pc)

### vp_type gives the type of values of the viewpoint:

In [6]:
vp_type(v)

PC

### apply the viewpoint to a sequence as follows:

In [7]:
v(choral)

pc('A')

### vp_map turns a sequence of constituents in to a sequence of viewpoint values:

In [8]:
vp_map(v,choral)

10-element Vector{pc}:
 pc('A')
 pc('B')
 pc('A')
 pc('G')
 pc('G')
 pc('A')
 pc('B')
 pc('D')
 pc('G')
 pc('G')

### Viewpoints can be delayed, linked and derived by composition with arbitrary functions:

In [9]:
v2 = LinkedViewpoint(DelayedViewpoint(v,1),v)
vp_map(v2,choral)

9-element Vector{Tuple{pc, pc}}:
 (pc('B'), pc('A'))
 (pc('A'), pc('B'))
 (pc('G'), pc('A'))
 (pc('G'), pc('G'))
 (pc('A'), pc('G'))
 (pc('B'), pc('A'))
 (pc('D'), pc('B'))
 (pc('G'), pc('D'))
 (pc('G'), pc('G'))

# Create Model 
The following creates an HGramModel of the choral using the pitch class viewpoint. 

In [10]:
using Models
m = HGramModel{pc,5}(vp_map(v,choral))

HGramModel{pc, 5}(Dict{Vector{pc}, Int64}([pc('B'), pc('A'), pc('G'), pc('G'), pc('A')] => 1, [pc('G'), pc('A')] => 1, [pc('G'), pc('G'), pc('A'), pc('B'), pc('D')] => 1, [pc('A'), pc('B'), pc('A')] => 1, [pc('G')] => 4, [pc('G'), pc('A'), pc('B'), pc('D')] => 1, [pc('B'), pc('D'), pc('G'), pc('G')] => 1, [pc('G'), pc('A'), pc('B'), pc('D'), pc('G')] => 1, [pc('A'), pc('G')] => 1, [pc('G'), pc('G'), pc('A')] => 1…), Set(pc[pc('D'), pc('B'), pc('G'), pc('A')]))

### The model elements is a set containing the pitch classes encountered: 

In [11]:
m.elems

Set{pc} with 4 elements:
  pc('D')
  pc('B')
  pc('G')
  pc('A')

### The model database maps sequences to their number of occurences:

In [12]:
m.db

Dict{Vector{pc}, Int64} with 32 entries:
  [pc('B'), pc('A'), pc('G'), pc('G'), pc('A')] => 1
  [pc('G'), pc('A')]                            => 1
  [pc('G'), pc('G'), pc('A'), pc('B'), pc('D')] => 1
  [pc('A'), pc('B'), pc('A')]                   => 1
  [pc('G')]                                     => 4
  [pc('G'), pc('A'), pc('B'), pc('D')]          => 1
  [pc('B'), pc('D'), pc('G'), pc('G')]          => 1
  [pc('G'), pc('A'), pc('B'), pc('D'), pc('G')] => 1
  [pc('A'), pc('G')]                            => 1
  [pc('G'), pc('G'), pc('A')]                   => 1
  [pc('G'), pc('G'), pc('A'), pc('B')]          => 1
  [pc('A'), pc('B')]                            => 2
  [pc('A'), pc('B'), pc('D'), pc('G'), pc('G')] => 1
  [pc('B'), pc('A'), pc('G'), pc('G')]          => 1
  [pc('A'), pc('B'), pc('D')]                   => 1
  [pc('D'), pc('G'), pc('G')]                   => 1
  [pc('G'), pc('A'), pc('B')]                   => 1
  [pc('B'), pc('D')]                            => 1
  [pc

# Maximum likelihood
The function ml computes the maximum likelihood of an element occuring in a given contexts from the model.

In [13]:
ml(pc('G'),m,m.elems)

0.4

In [14]:
ml(pc('E'),m,m.elems)

0.0

In [15]:
ml(pc('G'),[pc('G')],m,m.elems)

0.5

In [16]:
ml(pc('D'),[pc('B'),pc('A')],m,m.elems)

0.0

# Smoothed Prediction

The ppm function gives the probability of the symbol in a context from a given model and alphabet. The first two arguments specify the type of blending (backoff or interpolated) and the escape method (A, B, C, D or AX). 

## Backoff Smoothing

In [17]:
ppm(Backoff(),A(),pc('G'),[pc('G')],m,m.elems)

0.25

In [18]:
ppm(Backoff(),B(),pc('G'),[pc('G')],m,m.elems)

0.25

In [19]:
ppm(Backoff(),C(),pc('G'),[pc('G')],m,m.elems)

0.25

In [20]:
ppm(Backoff(),D(),pc('G'),[pc('G')],m,m.elems)

0.375

In [21]:
ppm(Backoff(),AX(),pc('G'),[pc('G')],m,m.elems)

0.16666666666666666

## Interpolated Smoothing

In [22]:
ppm(Interpolated(),A(),pc('G'),[pc('G')],m,m.elems)

0.44

In [23]:
ppm(Interpolated(),B(),pc('G'),[pc('G')],m,m.elems)

0.6

In [24]:
ppm(Interpolated(),C(),pc('E'),[pc('G')],m,m.elems)

0.039999999999999994

In [25]:
ppm(Interpolated(),D(),pc('G'),[pc('G')],m,m.elems)

0.5333333333333333

In [26]:
ppm(Interpolated(),AX(),pc('G'),[pc('G')],m,m.elems)

0.44

# Entropy
A Predictor contains all information required to make predictions: smoothing method, escape method, HGram model and alphabet. 

In [27]:
p = Predictor(Backoff(),A(),m,m.elems)

Predictor{pc}(Backoff(), A(), HGramModel{pc, 5}(Dict{Vector{pc}, Int64}([pc('B'), pc('A'), pc('G'), pc('G'), pc('A')] => 1, [pc('G'), pc('A')] => 1, [pc('G'), pc('G'), pc('A'), pc('B'), pc('D')] => 1, [pc('A'), pc('B'), pc('A')] => 1, [pc('G')] => 4, [pc('G'), pc('A'), pc('B'), pc('D')] => 1, [pc('B'), pc('D'), pc('G'), pc('G')] => 1, [pc('G'), pc('A'), pc('B'), pc('D'), pc('G')] => 1, [pc('A'), pc('G')] => 1, [pc('G'), pc('G'), pc('A')] => 1…), Set(pc[pc('D'), pc('B'), pc('G'), pc('A')])), Set(pc[pc('D'), pc('B'), pc('G'), pc('A')]))

A distribution contains a predictor and a context sequence. A distribution can be applied to an argument to obtain the probability. 

In [28]:
d = Distribution(p,[pc('G')])
d(pc('G'))

0.25

The information content of a symbol in a given context is calculated from a distribution. 

In [29]:
information_content(pc('G'),d)

2.0

The entropy of a distribution is calculated as follows:

In [30]:
entropy(d)

1.3537027664653143

In [31]:
entropy_max(d)

2.0

In [32]:
redundancy(d)

0.32314861676734286

# NEXT
- Check the maths is correct. Brackets!
- Should entropy divide by alphabet length?
- Combining predictors
- Combining distributions with different predictors.